In [8]:
import pandas as pd
import os

rsrc_folder = "./raw_files/"
op_folder = "./OP/"

input_df = pd.read_excel(os.path.join(rsrc_folder, "PROTECH CALIBER NOVEMBER.xlsx"),engine="openpyxl" , dtype={"LICENSE PLATE":"object","TRANSPONDER":"object","EXIT DATE/TIME":"datetime64[ns]","AMOUNT":"float64","POST DATE/TIME":"datetime64[ns]"})
master_abbv = pd.read_excel(os.path.join(rsrc_folder, "complete-agencys.xlsx"))
master_protech = pd.read_excel(os.path.join(rsrc_folder, "protech_lp-transponder_masterfile2.xlsx"),engine="openpyxl" , dtype={"State":"object","Transp. ID":"object","LP":"object","Transp. #":"object"})
master_caliber = pd.read_excel(os.path.join(rsrc_folder, "caliber_lp_masterfile_nov_23.xlsx"), dtype={"Cost Center":"object", "Transp. #":"object", "LP":"object"})

In [9]:
# normalize agencies
print("Normalizing Agency Names")
agencies = input_df['AGENCY'].drop_duplicates().to_list()

for agency in agencies:
    lookup_df = master_abbv[master_abbv['ABBV'] == agency]
    if lookup_df.shape[0] != 0:
        for m_i, m_row in lookup_df.iterrows():
            input_df.loc[input_df['AGENCY'] == agency, ["AGENCY"]] = str(m_row['AGENCY'])
            break

Normalizing Agency Names


In [10]:
print("splitting ntta location")
select_df = input_df[input_df['EXIT LANE'].isnull()]
locations = select_df['EXIT LOCATION'].drop_duplicates().to_list()

for location in locations:
    loc = location.split(" - ")[-1]
    lane = " - ".join(location.split(" - ")[:-1])
    input_df.loc[input_df['EXIT LOCATION'] == location, ["EXIT LANE"]] = lane
    input_df.loc[input_df['EXIT LOCATION'] == location, ["EXIT LOCATION"]] = loc

splitting ntta location


In [11]:
# handle protech
protech_df = input_df[input_df['CLIENT'] == "PROTECH"]

transponders = protech_df['TRANSPONDER'].drop_duplicates().to_list()
protech_df['unit'] = protech_df['cost center'] = "-"
master_protech = master_protech.astype({"State":"object","Transp. ID":"object","LP":"object","Transp. #":"object"})

for lp in transponders:
    lookup_df = master_protech[master_protech['Transp. #'] == lp]
    if lookup_df.shape[0] != 0:
        for m_i, m_row in lookup_df.iterrows():
            vehicle_id = str(m_row['Transp. ID']).split('-')
            if vehicle_id[1] != '':
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["unit"]] = vehicle_id[0]
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["cost center"]] = vehicle_id[1]
            else:
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["unit"]] = vehicle_id[0]
            if pd.isnull(m_row['LP']):
                pass
            else:
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
            protech_df.loc[protech_df['TRANSPONDER'] == lp, ["STATE"]] = str(m_row['State'])
            break
    else:
        lookup_df = master_protech[master_protech['Transp. ID'] == lp]
        if lookup_df.shape[0] != 0:
            for m_i, m_row in lookup_df.iterrows():
                vehicle_id = str(m_row['Transp. ID']).split('-')
                if vehicle_id[1] != '':
                    protech_df.loc[protech_df['TRANSPONDER'] == lp, ["unit"]] = vehicle_id[0]
                    protech_df.loc[protech_df['TRANSPONDER'] == lp, ["cost center"]] = vehicle_id[1]
                else:
                    protech_df.loc[protech_df['TRANSPONDER'] == lp, ["unit"]] = vehicle_id[0]
                if pd.isnull(m_row['LP']):
                    pass
                else:
                    protech_df.loc[protech_df['TRANSPONDER'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["STATE"]] = str(m_row['State'])
                break

transponders = protech_df['LICENSE PLATE'].drop_duplicates().to_list()
for lp in transponders:
    lookup_df = master_protech[master_protech['LP'] == lp]
    if lookup_df.shape[0] != 0:
        for m_i, m_row in lookup_df.iterrows():
            vehicle_id = str(m_row['Transp. ID']).split('-')
            if vehicle_id[1] != '':
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["cost center"]] = vehicle_id[1]
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
            break

select_df = protech_df[protech_df['LICENSE PLATE'] == "-"]
select_df.drop_duplicates(subset=['TRANSPONDER'], inplace=True)

for i_i, i_row in select_df.iterrows():
    protech_df.loc[protech_df['TRANSPONDER'] == i_row['TRANSPONDER'], ["LICENSE PLATE"]] = i_row['TRANSPONDER']

protech_df = protech_df.drop(columns=['CLIENT', 'TRANSPONDER'])
writer = pd.ExcelWriter(os.path.join(op_folder, f'protech_monthly_report.xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
protech_df.to_excel(writer, index=False)
writer.save()

protech_df['AMOUNT'] = protech_df['AMOUNT']*1.31
writer = pd.ExcelWriter(os.path.join(op_folder, f'protech_monthly_report_higher.xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
protech_df.to_excel(writer, index=False)
writer.save()

print("done")

C:\Users\GATHON~1\AppData\Local\Temp/ipykernel_11412/2558608390.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protech_df['unit'] = protech_df['cost center'] = "-"
C:\Users\GathoniNgambi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\GathoniNgambi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A va

done


In [12]:
# handle caliber
protech_df = input_df[input_df['CLIENT'] == "CALIBER"]
        
lps = protech_df['TRANSPONDER'].drop_duplicates().to_list()

for lp in lps:
    lookup_df = master_caliber[master_caliber['Transp. #'] == lp]
    if lookup_df.shape[0] != 0:
        print(lp)
        for m_i, m_row in lookup_df.iterrows():
            if pd.isnull(m_row['State']):
                pass
            else:
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["STATE"]] = str(m_row['State'])
            if pd.isnull(m_row['LP']):
                pass
            else:
                protech_df.loc[protech_df['TRANSPONDER'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
            # protech_df.loc[protech_df['TRANSPONDER'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
            break

lps = protech_df['LICENSE PLATE'].drop_duplicates().to_list()
protech_df['cost center'] = "-"

for lp in lps:
    lookup_df = master_caliber[master_caliber['Transp. #'] == lp]
    if lookup_df.shape[0] != 0:
        for m_i, m_row in lookup_df.iterrows():
            if pd.isnull(m_row['Cost Center']):
                pass
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["cost center"]] = str(m_row['Cost Center'])
            if pd.isnull(m_row['State']):
                pass
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["STATE"]] = str(m_row['State'])
            protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
            break
    else:
        lookup_df = master_caliber[master_caliber['LP'] == lp]
        if lookup_df.shape[0] != 0:
            for m_i, m_row in lookup_df.iterrows():
                if pd.isnull(m_row['Cost Center']):
                    pass
                else:
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["cost center"]] = str(m_row['Cost Center'])
                break
        else:
            pass

select_df = protech_df[protech_df['LICENSE PLATE'] == "-"]
select_df.drop_duplicates(subset=['TRANSPONDER'], inplace=True)

for i_i, i_row in select_df.iterrows():
    protech_df.loc[protech_df['TRANSPONDER'] == i_row['TRANSPONDER'], ["LICENSE PLATE"]] = i_row['TRANSPONDER']
    
protech_df = protech_df.drop(columns=['CLIENT', 'TRANSPONDER'])
writer = pd.ExcelWriter(os.path.join(op_folder, f'caliber_monthly_report.xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
protech_df.to_excel(writer, index=False)
writer.save()

protech_df['AMOUNT'] = protech_df['AMOUNT']*1.31
writer = pd.ExcelWriter(os.path.join(op_folder, f'caliber_monthly_report_higher.xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
protech_df.to_excel(writer, index=False)
writer.save()

print("done")

DFW.01574785
DFW.02253637
DFW.01574787
DFW.01574786
DFW.02253679
DFW.01582607
DFW.01574788
DFW.03650577
DFW.02253627
DFW.01593788
DFW.02253681
DFW.03352873
DFW.02253680
DFW.01582608
DFW.01582609
DFW.02110950
DFW.02110952
DFW.02110951
10300036124943
10300037588773
10300036125007
10300037588567
10300037588971
10300036124919
10300036594343
10300037588278
10300037588674
10300036594350
10300036594301
10300037588369
5785855
5785856
5785857
6020916
6020917
6020918
6020919


C:\Users\GathoniNgambi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\GATHON~1\AppData\Local\Temp/ipykernel_11412/1302149062.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protech_df['cost center'] = "-"
C:\Users\GathoniNgambi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be 

done


In [11]:
# protech_df = input_df[input_df['CLIENT'] == "CALIBER"]
        
# lps = protech_df['LICENSE PLATE'].drop_duplicates().to_list()

# for lp in lps:
#     lookup_df = master_caliber[master_caliber['LP'] == lp]
#     if lookup_df.shape[0] != 0:
#         print(lp)
#         for m_i, m_row in lookup_df.iterrows():
#             if pd.isnull(m_row['Cost Center']):
#                 pass
#             else:
#                 protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["COST CENTER"]] = str(m_row['Cost Center'])
#             if pd.isnull(m_row['Unit #']):
#                 pass
#             else:
#                 protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["UNIT #"]] = str(m_row['Unit #'])
#             # protech_df.loc[protech_df['TRANSPONDER'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
#             break

import pandas as pd
import os

rsrc_folder = "./raw_files/"
op_folder = "./OP/"

protech_df = pd.read_excel(os.path.join(rsrc_folder, "Caliber_November_Trxn_Data (1).xlsx"),engine="openpyxl" , dtype={"LICENSE PLATE":"object","TRANSPONDER":"object","EXIT DATE/TIME":"datetime64[ns]","AMOUNT":"float64","POST DATE/TIME":"datetime64[ns]"})
# master_abbv = pd.read_excel(os.path.join(rsrc_folder, "complete-agencys.xlsx"))
# master_protech = pd.read_excel(os.path.join(rsrc_folder, "protech_lp-transponder_masterfile2.xlsx"),engine="openpyxl" , dtype={"State":"object","Transp. ID":"object","LP":"object","Transp. #":"object"})
master_caliber = pd.read_excel(os.path.join(rsrc_folder, "caliber_lp_masterfile_nov_23.xlsx"), dtype={"Cost Center":"object", "Transp. #":"object", "LP":"object"})


lps = protech_df['LICENSE PLATE'].drop_duplicates().to_list()
# protech_df['cost center'] = "-"

for lp in lps:
    lookup_df = master_caliber[master_caliber['Transp. #'] == lp]
    if lookup_df.shape[0] != 0:
        for m_i, m_row in lookup_df.iterrows():
            if pd.isnull(m_row['Cost Center']):
                pass
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["COST CENTER"]] = str(m_row['Cost Center'])
            if pd.isnull(m_row['Unit #']):
                pass
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["UNIT #"]] = str(m_row['Unit #'])
            if pd.isnull(m_row['State']):
                pass
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["STATE"]] = str(m_row['State'])
            protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
            break
    else:
        lookup_df = master_caliber[master_caliber['LP'] == lp]
        if lookup_df.shape[0] != 0:
            for m_i, m_row in lookup_df.iterrows():
                if pd.isnull(m_row['Cost Center']):
                    pass
                else:
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["COST CENTER"]] = str(m_row['Cost Center'])
                if pd.isnull(m_row['Unit #']):
                    pass
                else:
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["UNIT #"]] = str(m_row['Unit #'])
                    break
        else:
            pass

# select_df = protech_df[protech_df['LICENSE PLATE'] == "-"]
# select_df.drop_duplicates(subset=['TRANSPONDER'], inplace=True)

# for i_i, i_row in select_df.iterrows():
#     protech_df.loc[protech_df['TRANSPONDER'] == i_row['TRANSPONDER'], ["LICENSE PLATE"]] = i_row['TRANSPONDER']
    
# protech_df = protech_df.drop(columns=['CLIENT', 'TRANSPONDER'])
writer = pd.ExcelWriter(os.path.join(op_folder, f'Caliber_November_Trxn_Data (1).xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
protech_df.to_excel(writer, index=False)
writer.save()

# protech_df['AMOUNT'] = protech_df['AMOUNT']*1.31
# writer = pd.ExcelWriter(os.path.join(op_folder, f'caliber_monthly_report_higher.xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
# protech_df.to_excel(writer, index=False)
# writer.save()

print("done")

done


In [10]:
import pandas as pd
import os

rsrc_folder = "./raw_files/"
op_folder = "./OP/"

protech_df = pd.read_excel(os.path.join(rsrc_folder, "Protech_November_Trxn_Data.xlsx"),engine="openpyxl" , dtype={"LICENSE PLATE":"object","TRANSPONDER":"object","EXIT DATE/TIME":"datetime64[ns]","AMOUNT":"float64","POST DATE/TIME":"datetime64[ns]"})
# master_abbv = pd.read_excel(os.path.join(rsrc_folder, "complete-agencys.xlsx"))
master_protech = pd.read_excel(os.path.join(rsrc_folder, "protech_lp-transponder_masterfile2.xlsx"),engine="openpyxl" , dtype={"State":"object","Transp. ID":"object","LP":"object","Transp. #":"object"})
# master_caliber = pd.read_excel(os.path.join(rsrc_folder, "caliber_lp_masterfile_nov_23.xlsx"), dtype={"Cost Center":"object", "Transp. #":"object", "LP":"object"})

transponders = protech_df['LICENSE PLATE'].drop_duplicates().to_list()
protech_df['unit'] = protech_df['cost center'] = "-"
master_protech = master_protech.astype({"State":"object","Transp. ID":"object","LP":"object","Transp. #":"object"})

for lp in transponders:
    lookup_df = master_protech[master_protech['Transp. #'] == lp]
    if lookup_df.shape[0] != 0:
        for m_i, m_row in lookup_df.iterrows():
            vehicle_id = str(m_row['Transp. ID']).split('-')
            if vehicle_id[1] != '':
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["cost center"]] = vehicle_id[1]
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
            if pd.isnull(m_row['LP']):
                pass
            else:
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
            protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["STATE"]] = str(m_row['State'])
            break
    else:
        lookup_df = master_protech[master_protech['Transp. ID'] == lp]
        if lookup_df.shape[0] != 0:
            for m_i, m_row in lookup_df.iterrows():
                vehicle_id = str(m_row['Transp. ID']).split('-')
                if vehicle_id[1] != '':
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["cost center"]] = vehicle_id[1]
                else:
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
                if pd.isnull(m_row['LP']):
                    pass
                else:
                    protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["LICENSE PLATE"]] = str(m_row['LP'])
                protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["STATE"]] = str(m_row['State'])
                break
        else:
            lookup_df = master_protech[master_protech['LP'] == lp]
            if lookup_df.shape[0] != 0:
                for m_i, m_row in lookup_df.iterrows():
                    vehicle_id = str(m_row['Transp. ID']).split('-')
                    if vehicle_id[1] != '':
                        protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]
                        protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["cost center"]] = vehicle_id[1]
                    else:
                        protech_df.loc[protech_df['LICENSE PLATE'] == lp, ["unit"]] = vehicle_id[0]


writer = pd.ExcelWriter(os.path.join(op_folder, f'Protech_November_Trxn_Data.xlsx'), engine='xlsxwriter', datetime_format='m/dd/yyyy hh:mm:ss')
protech_df.to_excel(writer, index=False)
writer.save()

print("done")

done
